In [1]:
import os
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import matplotlib.pylab as plb
import cv2
import pandas as pd

In [2]:
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend" #for GPU Support 

In [3]:
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [4]:
currentDir = os.getcwd()
datasetDir = currentDir + "/ImgDir/"
trainDir = os.path.join(datasetDir, "train")
testDir = os.path.join(datasetDir, "test")
validDir = os.path.join(datasetDir, "valid")
y_dataDir = os.path.join(datasetDir, "y_data")

In [5]:
def sortFirst(val):
    return val[0]

def getBinary(dataFile):
    listOfTraining = []
    trainingHeader = []
    with open(dataFile) as csvfile:
#     reader = csv.DictReader(csvfile)
        reader2 = csv.reader(csvfile)
        listOfTraining = list(reader2)
        trainingHeader = listOfTraining[0]
        listOfTraining.pop(0)
#         listOfTraining = listOfTraining.sort(key = sortFirst, reverse = False)
#         np.asarray(listofTraining, dtype=np.int32)
#         return np.asarray(listofTraining, dtype=np.int32)
    listOfTrainingBinary = []
    for item in listOfTraining:
        listOfTrainingBinary.append(item[1])
    return np.asarray(listOfTrainingBinary, dtype=np.int32)

In [6]:
y_train = []
y_trainDir = os.path.join(y_dataDir, 'train_split_Depression_AVEC2017-edited.csv')
# print(y_trainDir)
y_train = getBinary(y_trainDir)
y_testDir = os.path.join(y_dataDir, 'dev_split_Depression_AVEC2017.csv')
# print(y_testDir)
y_test = getBinary(y_testDir)
print(y_train)
print(y_test)

# Y_train = np.asarray(y_train, dtype=np.int32)

[0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1
 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0
 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0]


In [7]:
x_trainDir = trainDir
x_testDir = testDir

In [7]:
x_trainDir

'/Users/aaron/Desktop/FYP Output/Keras/ImgDir/train'

In [8]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation

from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import tensorflow as tf

Using plaidml.keras.backend backend.


In [9]:
input_shape = (101, 1000, 3)

# model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape=input_shape))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())
# model.add(Dense(64))
# model.add(Activation('relu'))
# # model.add(Dropout(0.5))
# model.add(Dense(1))
# model.add(Activation('sigmoid'))

# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

# model.summary()


# TAKEN FROM DEPRESSION DETECT
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='valid', strides=1,
                 input_shape=input_shape, activation='relu'))

model.add(MaxPooling2D(pool_size=(4, 3), strides=(1, 3)))

model.add(Conv2D(32, (1, 3), padding='valid', strides=1,
          input_shape=input_shape, activation='relu'))

model.add(MaxPooling2D(pool_size=(1, 3), strides=(1, 3)))

model.add(Flatten())
# model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('softmax'))
# model.add(Activation(tf.nn.softmax)) #fixes axis error for softmax

# model.compile(loss='sparse_categorical_crossentropy',
#              optimizer='adadelta',
#              metrics=['accuracy'])

# model.compile(loss='categorical_crossentropy',
#               optimizer='adadelta',
#               metrics=['accuracy'])

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

INFO:plaidml:Opening device "metal_intel(r)_hd_graphics_630.0"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 99, 998, 32)       896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 96, 332, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 330, 32)       3104      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 96, 110, 32)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 337920)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               173015552 
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
__________

In [10]:
from keras.preprocessing.image import ImageDataGenerator

In [11]:
train_datagen = ImageDataGenerator()

# train_datagen = keras.utils.Sequence()

train_generator = train_datagen.flow_from_directory(
        x_trainDir,
        target_size=(101, 1000),
        batch_size=32,
        class_mode='binary')

validation_generator = train_datagen.flow_from_directory(
        x_testDir,
        target_size=(101, 1000),
        batch_size=32,
        class_mode='binary')

Found 9320 images belonging to 103 classes.
Found 3587 images belonging to 35 classes.


In [12]:
#Changes the class values generated from the generator
def updateBinary(generator, y_binary):
    counter = 0
    for k,v in generator.class_indices.items():
        generator.class_indices[k] = y_binary[counter]
        counter += 1

# counter = 0
# for k,v in train_generator.class_indices.items():
#     train_generator.class_indices[k] = y_train[counter]
#     counter += 1        

In [46]:
# for k,v in train_generator.class_indices.items():
#     print(k,v)

In [13]:
for item in validation_generator.class_indices.items():
    print(item)

('302_AUDIO', 0)
('307_AUDIO', 1)
('331_AUDIO', 2)
('335_AUDIO', 3)
('346_AUDIO', 4)
('367_AUDIO', 5)
('377_AUDIO', 6)
('381_AUDIO', 7)
('382_AUDIO', 8)
('388_AUDIO', 9)
('389_AUDIO', 10)
('390_AUDIO', 11)
('395_AUDIO', 12)
('403_AUDIO', 13)
('404_AUDIO', 14)
('406_AUDIO', 15)
('413_AUDIO', 16)
('417_AUDIO', 17)
('418_AUDIO', 18)
('420_AUDIO', 19)
('422_AUDIO', 20)
('436_AUDIO', 21)
('439_AUDIO', 22)
('440_AUDIO', 23)
('451_AUDIO', 24)
('458_AUDIO', 25)
('472_AUDIO', 26)
('476_AUDIO', 27)
('477_AUDIO', 28)
('482_AUDIO', 29)
('483_AUDIO', 30)
('484_AUDIO', 31)
('489_AUDIO', 32)
('490_AUDIO', 33)
('492_AUDIO', 34)


In [14]:
#updates each folder with the corresponding binary for depression/non-depression cases
updateBinary(train_generator, y_train)
updateBinary(validation_generator, y_test)

In [15]:
# for k,v in train_generator.class_indices.items():
#     print(k,v)

In [15]:
#Saving Model
from keras.callbacks import ModelCheckpoint

model_path = os.path.join(currentDir,'DepressionAnalysisModel3.h5')

# checkpoint = ModelCheckpoint(model_path, monitor='loss', verbose=1, save_best_only=True, mode='max')

# checkpoint = ModelCheckpoint(model_path, monitor='acc', verbose=1, save_best_only=True, mode='max')

checkpoint = ModelCheckpoint(model_path, monitor='acc', verbose=1, save_best_only=False, save_weights_only=False, mode='auto')

callbacks_list = [checkpoint]

In [1]:
# input_shape

NameError: name 'input_shape' is not defined

In [45]:
# model.fit_generator(
#         train_generator,
#         steps_per_epoch=2000,
#         epochs=50,
#         validation_data=validation_generator,
#         validation_steps=800)

numOfSteps = 1000
numOfEpoch = 7
numOfValidation = 100
batchSize = 32

model.fit_generator(
        train_generator,
        steps_per_epoch=numOfSteps,
        callbacks=callbacks_list,
#         batch_size=batchSize
        epochs=numOfEpoch,
        validation_data=validation_generator,
        validation_steps=numOfValidation)

Epoch 1/7
1000/1000 [==============================] - 2442s 2s/step - loss: -805.7671 - acc: 0.0083 - val_loss: -251.8996 - val_acc: 0.0341

Epoch 00001: saving model to /Users/aaron/Desktop/FYP Output/Keras/DepressionAnalysisModel2.h5
Epoch 2/7
1000/1000 [==============================] - 2329s 2s/step - loss: -805.4107 - acc: 0.0088 - val_loss: -252.5593 - val_acc: 0.0328

Epoch 00002: saving model to /Users/aaron/Desktop/FYP Output/Keras/DepressionAnalysisModel2.h5
Epoch 3/7
1000/1000 [==============================] - 2271s 2s/step - loss: -805.3302 - acc: 0.0086 - val_loss: -251.2371 - val_acc: 0.0331

Epoch 00003: saving model to /Users/aaron/Desktop/FYP Output/Keras/DepressionAnalysisModel2.h5
Epoch 4/7
1000/1000 [==============================] - 2198s 2s/step - loss: -805.9506 - acc: 0.0081 - val_loss: -247.5720 - val_acc: 0.0369

Epoch 00004: saving model to /Users/aaron/Desktop/FYP Output/Keras/DepressionAnalysisModel2.h5
Epoch 5/7
1000/1000 [==============================]

In [16]:
# model.fit_generator(
#         train_generator,
#         steps_per_epoch=2000,
#         epochs=50,
#         validation_data=validation_generator,
#         validation_steps=800)

numOfSteps = 1000
numOfEpoch = 7
numOfValidation = 100
batchSize = 32

model.fit_generator(
        train_generator,
        steps_per_epoch=numOfSteps,
        callbacks=callbacks_list,
#         batch_size=batchSize
        epochs=numOfEpoch,
        validation_data=validation_generator,
        validation_steps=numOfValidation)

Epoch 1/7
   4/1000 [..............................] - ETA: 1:37:48 - loss: -783.2942 - acc: 0.0078

KeyboardInterrupt: 

In [23]:
# model.summary()

In [47]:
from keras.models import load_model

model.save(os.path.join(currentDir,'DepressionAnalysisModel2 after 7 epoch.h5'))  # creates a HDF5 file
#del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
#model = load_model(os.join.path(currentDir,'my_model.h5'))